# GRIDMET example

The ETo/ETr calculation for recent GRIDMET images have been updated to follow the equations in ASCE 2005.

Test that the precomputed ETr in the collection matches wih the ETr generated using the eerefet module. 

https://code.earthengine.google.com/d2c73645498e95e0ce43671effceca66

In [1]:
import math
import ee
import refetgee

ee.Initialize()

#### Load GRIDMET for a single date

In [2]:
gridmet_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')\
    .filterDate('2018-04-01', '2018-04-02')
    
# gridmet_img = ee.Image(ee.ImageCollection(gridmet_coll.first()))
gridmet_img = ee.Image('IDAHO_EPSCOR/GRIDMET/20180401')

#### Ancillary data and test point

In [3]:
test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match GRIDMET elevation grid exactly
elev_img = ee.Image("projects/climate-engine/gridmet/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the GRIDMET images
crs = gridmet_img.projection().getInfo()['wkt']
geo = gridmet_img.projection().getInfo()['transform']

#### Precomputed ETr

In [4]:
etr = gridmet_img.select('etr').reduceRegion(
    ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etr.getInfo()['etr'])

7.627318382263184


#### Compute ETr for a single image

In [5]:
print(refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).etr()\
      .rename(['etr'])\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

7.624161658057224


#### Compute ETr for a collection in a mapped function

In [6]:
def gridmet_etr(img):
    return ee.Image(refetgee.Daily.gridmet(img, elev=elev_img, lat=lat_img).etr())\
        .rename(['etr'])\
        .copyProperties(img, ['system:time_start'])
etr = ee.Image(ee.ImageCollection(gridmet_coll.map(gridmet_etr)).first())\
    .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etr.getInfo()['etr'])

7.624161658057224
